# Segmenting Covid-19 Data into Regions and SubRegions

NorthEast Region:
New England (Connecticut, Maine, Massachusetts, New Hampshire, Rhode Island, and Vermont)



In [1]:
#import boto3
import pandas as pd
import numpy as np
import io
#from sagemaker import get_execution_role
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

# Amazon SageMaker Python SDK
# import sagemaker.amazon.common as smac 
# import sagemaker

# instruction to Jupyter to inline all the visualizations
%matplotlib inline 

In [2]:
# Load enigma dataset.
usa = pd.read_csv('enigma_Aug_8_2020.csv')
#usa.head(20)

# role = get_execution_role()
# input_bucket_states = 'enigma-data'
# in_data_key_states = 'enigma_covid_19/202008052100/enigma_covid_19-enigma_covid_19_us_states__202008052100__202008052100.csv'
# training_data_location_states = 's3://{}/{}'.format(input_bucket_states, in_data_key_states)

# # Define the headers since the data does not have any
# #headers = ['date','county','state','fips','cases','deaths']

# #df = pd.read_csv(training_data_location, names=headers)
# usa = pd.read_csv(training_data_location_states)
# print('uploaded training data from location: {}'.format(training_data_location_states))
usa.sample(20)

,state_fips,state_name,lat,long,date,cases,deaths,tests_positive,tests_negative,tests_pending,tests,patients_icu,patients_hosp,patients_vent,recovered
4814,13,Georgia,32.629579,-83.423511,4/8/2020,10204,370,9901.0,28886.0,NaN,38787.0,NaN,NaN,NaN,NaN
1014,44,Rhode Island,41.597419,-71.527272,6/23/2020,16533,906,16612.0,128896.0,NaN,145508.0,19.0,105.0,17.0,1562.0
4655,12,Florida,28.457430,-82.409148,3/29/2020,4943,60,4246.0,39070.0,NaN,43316.0,NaN,NaN,NaN,NaN
1712,49,Utah,39.334992,-111.656333,4/27/2020,4236,41,4233.0,95962.0,NaN,100195.0,NaN,NaN,NaN,1641.0
4419,10,Delaware,38.998566,-75.441644,5/26/2020,9066,335,9066.0,45848.0,NaN,54914.0,NaN,201.0,NaN,4802.0
6177,23,Maine,45.409284,-68.666616,5/13/2020,1515,66,1515.0,22092.0,NaN,23607.0,20.0,41.0,7.0,943.0
7383,31,Nebraska,41.543305,-99.811865,5/23/2020,11988,150,11662.0,68320.0,NaN,79982.0,NaN,NaN,NaN,NaN
1644,48,Texas,31.434703,-99.281824,7/23/2020,376104,4714,361125.0,2803531.0,NaN,3164656.0,3281.0,10893.0,NaN,203826.0
5108,16,Idaho,44.348422,-114.558854,4/19/2020,1672,45,1668.0,15773.0,NaN,17441.0,NaN,43.0,NaN,520.0
2165,53,Washington,47.407324,-120.575800,4/2/2020,6586,272,9201.0,86091.0,NaN,95292.0,NaN,NaN,NaN,NaN


In [3]:
# drop unneeded colums
usa = usa.drop(columns = ["state_fips", "lat", "long"])

In [4]:
# Convert to Datetime
usa['datetime'] = pd.to_datetime(usa['date'])
# usa["day"] = usa['datetime'].map(lambda x: x.day)
# usa["month"] = usa['datetime'].map(lambda x: x.month)
# usa["year"] = usa['datetime'].map(lambda x: x.year)
# usa.sample(5)

In [ ]:
usa_group = usa.groupby(["datetime"], as_index = False).sum()
usa_group.sample(5)

In [ ]:
# Separate out Datetime

usa_group["day"] = usa_group['datetime'].map(lambda x: x.day)
usa_group["month"] = usa_group['datetime'].map(lambda x: x.month)
usa_group["year"] = usa_group['datetime'].map(lambda x: x.year)
usa_group.sample(5)

In [ ]:
usa_group["growth_rate"] = usa_group["cases"].pct_change()
usa_group["growth_rate"].max()

In [ ]:
usa_group["inc_dec"] = 0
usa_group.head()

In [ ]:
# 0 is for no growth
# 1 is for growth less than 10%
# 2 is for growth equal or more than 10%
# 3 is for decrease less than 10%
# 4 is for decrease equal or more than 10%

i = 0
while i < ((len(usa_group.index))-1):
    if ((usa_group.iloc[i+1, 1] - usa_group.iloc[i, 1]) / usa_group.iloc[i, 1]) == 0:
        usa_group.iloc[i+1, 14] = 0
        
    elif (((usa_group.iloc[i+1, 1] - usa_group.iloc[i, 1]) / usa_group.iloc[i, 1]) < 0.10) and \
    (((usa_group.iloc[i+1, 1] - usa_group.iloc[i, 1]) / usa_group.iloc[i, 1]) > 0):
        usa_group.iloc[i+1, 14] = 1
        
    elif ((usa_group.iloc[i+1, 1] - usa_group.iloc[i, 1]) / usa_group.iloc[i, 1]) >= 0.10:
        usa_group.iloc[i+1, 14] = 2
        
    elif (((usa_group.iloc[i+1, 1] - usa_group.iloc[i, 1]) / usa_group.iloc[i, 1]) > -0.10) and \
    (((usa_group.iloc[i+1, 1] - usa_group.iloc[i, 1]) / usa_group.iloc[i, 1]) < 0):
        usa_group.iloc[i+1, 14] = 3
    elif (((usa_group.iloc[i+1, 1] - usa_group.iloc[i, 1]) / usa_group.iloc[i, 1]) <= -0.10):
        usa_group.iloc[i+1, 14] = 4
        
    i += 1

In [ ]:
# usa_group.loc[usa_group["inc_dec"]==0].count()       # 21
# usa_group.loc[usa_group["inc_dec"]==1].count()        # 123
# usa_group.loc[usa_group["inc_dec"]==2].count()          # 46

NorthEast Region:
New England Divsion 1 (Connecticut, Maine, Massachusetts, New Hampshire, Rhode Island, and Vermont)

In [ ]:
# Created DF based New England Divsion
new_england = usa[(usa['state_name']=="Connecticut") | (usa['state_name'] == 'Maine') |
                      (usa['state_name'] == 'Massachusetts') | (usa['state_name'] == 'New Hampshire') | 
                     (usa['state_name']=='Rhode Island')| (usa['state_name'] =='Vermont')]
new_england.head()

NorthEast Region
Mid-Atlantic Divsion 2 (New Jersey, New York, and Pennsylvania)

In [ ]:
# Create DF based on Mid Atlantic Divsion
mid_atlantic = usa[(usa['state_name']=="New Jersey") | (usa['state_name'] == 'New York') |
                      (usa['state_name'] == 'Pennsylvania')]

mid_atlantic.head()

Midwest Region
Division 3: East North Central (Illinois, Indiana, Michigan, Ohio, and Wisconsin)

In [ ]:
# Created DF based New Englend Divsion
east_north_central = usa[(usa['state_name']=='Illinois') | (usa['state_name'] == 'Indiana') |
                      (usa['state_name'] == 'Michigan') | (usa['state_name'] == 'Ohio') | \
                     (usa['state_name']=='Wisconsin')]
east_north_central.head()

 West North Central Divsion 4 (Iowa, Kansas, Minnesota, Missouri, Nebraska, North Dakota, and South Dakota)

In [ ]:
# Creat DF for just West North Central
west_northern_central =  usa[(usa['state_name']=='Iowa') | (usa['state_name'] == 'Kansas') |
                      (usa['state_name'] == 'Minnesota') | (usa['state_name'] == 'Missouri') | 
                     (usa['state_name']=='North Dakota') | (usa['state_name']=='South Dakota')] 

west_northern_central.head()
                    

In [ ]:
# Create Function
def make_regions(a,b,c,d,e,f)

 South Division 5: South Atlantic (Delaware, Florida, Georgia, Maryland, North Carolina, South Carolina, Virginia, District of Columbia, and West Virginia)

In [ ]:
# Make Df for just South Atlantic divsion.
south_atlantic = usa[(usa['state_name']=='Delaware') | (usa['state_name'] == 'Flordia') |
                      (usa['state_name'] == 'Georgia') | (usa['state_name'] == 'Maryland') | 
                     (usa['state_name']=='North Carolina') | (usa['state_name']=='South Carolina') |
                     (usa['state_name']=='Virginia') | (usa['state_name']== 'District of Columbia') |
                    (usa['state_name'] == 'West Virginia')]
south_atlantic

South Division 6: East South Central (Alabama, Kentucky, Mississippi, and Tennessee)

In [ ]:
# Create DF for just East South Central Divsion
east_south_central =  usa[(usa['state_name']=='Alabama') | (usa['state_name'] == 'Kentucky') |
                      (usa['state_name'] == 'Mississippi') | (usa['state_name'] == 'Tennessee')] 

east_south_central.head()

South Division 7: West South Central (Arkansas, Louisiana, Oklahoma, and Texas)

In [ ]:
west_northern_central =  usa[(usa['state_name']=='Arkansas') | (usa['state_name'] == 'Louisiana') |
                      (usa['state_name'] == 'Oklahoma') | (usa['state_name'] == 'Texas') ] 

west_northern_central.head()

West 
Mountain Divsion 8 (Arizona, Colorado, Idaho, Montana, Nevada, New Mexico, Utah, and Wyoming)

In [ ]:
# Create a DF for just Mountain divsion
mountain =  usa[(usa['state_name']=='Arizona') | (usa['state_name'] == 'Colorado') |
                      (usa['state_name'] == 'Idaho') | (usa['state_name'] == 'Montana') | 
                     (usa['state_name']=='Nevada') | (usa['state_name']=='New Mexico') |
                     (usa['state_name']=='Utah') | (usa['state_name']== 'Wyoming') ]

mountain.head()

West Pacific (Alaska, California, Hawaii, Oregon, and Washington)

In [5]:
# Make DF for just Pacific Divsion
pacific =  usa[(usa['state_name']=='Alaska') | (usa['state_name']=='California') | (usa['state_name'] == 'Hawaii') |
                      (usa['state_name'] == 'Oregon') | (usa['state_name'] == 'Washington') ]

pacific.head()

,state_name,date,cases,deaths,tests_positive,tests_negative,tests_pending,tests,patients_icu,patients_hosp,patients_vent,recovered,datetime
603,Oregon,2/28/2020,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-28
604,Oregon,2/29/2020,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-29
605,Oregon,3/1/2020,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-01
606,Oregon,3/2/2020,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-02
607,Oregon,3/3/2020,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-03


In [ ]:
# Convert to Datetime
#pacific['datetime'] = pd.to_datetime(pacific['date'])

In [11]:
pacific_group = pacific.groupby(["datetime"], as_index = False).sum()
pacific_group.head(5)

,datetime,cases,deaths,tests_positive,tests_negative,tests_pending,tests,patients_icu,patients_hosp,patients_vent,recovered
0,2020-01-21,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-22,1,0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
2,2020-01-23,1,0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
3,2020-01-24,1,0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
4,2020-01-25,2,0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0


In [9]:
# Separate out Datetime

pacific_group["day"] = pacific_group['datetime'].map(lambda x: x.day)
pacific_group["month"] = pacific_group['datetime'].map(lambda x: x.month)
pacific_group["year"] = pacific_group['datetime'].map(lambda x: x.year)
pacific_group.tail(5)

,datetime,cases,deaths,tests_positive,tests_negative,tests_pending,tests,patients_icu,patients_hosp,patients_vent,recovered,day,month,year,growth
185,2020-07-24,516819,10243,505685.0,7959384.0,0.0,8465069.0,2342.0,9515.0,78.0,5466.0,24,7,2020,0.022705
186,2020-07-25,528166,10349,517133.0,8106070.0,0.0,8623203.0,2342.0,9527.0,84.0,5504.0,25,7,2020,0.021955
187,2020-07-26,535284,10381,526988.0,8244616.0,0.0,8771604.0,2342.0,9532.0,85.0,5525.0,26,7,2020,0.013477
188,2020-07-27,544665,10488,535162.0,8384812.0,0.0,8919974.0,2342.0,9517.0,72.0,5680.0,27,7,2020,0.017525
189,2020-07-28,554028,10699,542348.0,8524496.0,0.0,9066844.0,2256.0,9016.0,84.0,5763.0,28,7,2020,0.017190


In [ ]:
# add column for calculating the increase/decrease in cases
# pacific_group["growth_rate"] = 0
# pacific_group.head(5)

In [ ]:
# # 0 is for no growth
# # 1 is for growth less than 10%
# # 2 is for growth equal or more than 10%
# # 3 is for decrease less than 10%
# # 4 is for decrease equal or more than 10%

# i = 0
# while i < ((len(pacific_group.index))-1):
#     if ((pacific_group.iloc[i+1, 1] - pacific_group.iloc[i, 1]) / pacific_group.iloc[i, 1]) == 0:
#         pacific_group.iloc[i+1, 14] = 0
        
#     elif (((pacific_group.iloc[i+1, 1] - pacific_group.iloc[i, 1]) / pacific_group.iloc[i, 1]) < 0.10) and \
#     (((pacific_group.iloc[i+1, 1] - pacific_group.iloc[i, 1]) / pacific_group.iloc[i, 1]) > 0):
#         pacific_group.iloc[i+1, 14] = 1
        
#     elif ((pacific_group.iloc[i+1, 1] - pacific_group.iloc[i, 1]) / pacific_group.iloc[i, 1]) >= 0.10:
#         pacific_group.iloc[i+1, 14] = 2
        
#     elif (((pacific_group.iloc[i+1, 1] - pacific_group.iloc[i, 1]) / pacific_group.iloc[i, 1]) > -0.10) and \
#     (((pacific_group.iloc[i+1, 1] - pacific_group.iloc[i, 1]) / pacific_group.iloc[i, 1]) < 0):
#         pacific_group.iloc[i+1, 14] = 3
#     elif (((pacific_group.iloc[i+1, 1] - pacific_group.iloc[i, 1]) / pacific_group.iloc[i, 1]) <= -0.10):
#         pacific_group.iloc[i+1, 14] = 4
        
#     i += 1

# pacific_group.head(5)

In [8]:
pacific_group["growth"] = pacific_group["cases"].pct_change()
pacific_group["growth"].max()

1.0

In [ ]:
pacific["growth"] = pacific["cases"].pct_change()
pacific["growth"].max()

In [ ]:
pacific_group.loc[pacific_group.month==3].max()

In [ ]:

i = 0
while i < ((len(pacific_group.index))-1):
    pacific_group.iloc[i+1, 14] = ((pacific_group.iloc[i+1, 1] - pacific_group.iloc[i, 1]) / pacific_group.iloc[i, 1])

        
    i += 1

pacific_group.sample(10)

In [ ]:
pacific_group["growth_rate"].max()

In [ ]:
from sklearn.linear_model import LinearRegression
pacific_group = pacific_group.dropna()

In [ ]:
# X = pacific_group[["day", "month", "year", "cases","deaths","tests_positive","tests_negative","tests_pending","tests","patients_icu","patients_hosp", "patients_vent", "recovered"]]
# y = pacific_group["growth_rate"].values.reshape(-1,1)
# print(X.shape)
# print(y.shape)
# X
X = pacific_group[["day", "month", "year", "growth_rate"]]
y = pacific_group["cases"].values.reshape(-1,1)
print(X.shape)
print(y.shape)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data

X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

In [ ]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [ ]:
lr = LinearRegression()
lr.fit(X_train ,y_train)

In [ ]:
lr.score(X_test, y_test)

In [ ]:
def plot_corr(df, size=6):
    """
    Function plots a graphical correlation matrix for each pair of columns in the dataframe. 
    Shows the relationship between features.

    Input:
        df: pandas DataFrame
        size: vertical and horizontal size of the plot

    Displays:
        matrix of correlation between columns.  
                                                0 ------------------>  1
                                                Expect a line running from top left to bottom right
    """

    corr = df.corr()    # data frame correlation function
    fig, ax = plt.subplots(figsize=(size, size))
    ax.matshow(corr)   # color code the rectangles by correlation value
    plt.xticks(range(len(corr.columns)), corr.columns)  # draw x tick marks
    plt.yticks(range(len(corr.columns)), corr.columns)  # draw y tick marks

In [ ]:
plot_corr(pacific_group)

In [ ]:
from sklearn.model_selection import cross_val_score
# function to get cross validation scores
def get_cv_scores(model):
    scores = cross_val_score(lr,
                             X_train,
                             y_train,
                             cv=5,
                             scoring='r2')
    
    print('CV Mean: ', np.mean(scores))
    print('STD: ', np.std(scores))

In [ ]:
get_cv_scores(lr)

In [ ]:
# trying k-fold because data is imbalanced

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(n_splits=3)
kf.get_n_splits(X)

In [ ]:
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [ ]:
pacific_group.plot.line(y="growth_rate", x="datetime")